In [2]:
import pandas as pd
import numpy as np
import scipy.stats
import re
import statsmodels.stats.multitest
import operator
import cptac.utils as u

def wrap_ttest(df, label_column, comparison_columns=None, alpha=.05, return_all=False, correction_method='bonferroni', mincount=3, pval_return_corrected=True):
    try:
        '''Verify precondition that label column exists and has exactly 2 unique values'''
        label_values = df[label_column].unique()
        if len(label_values) != 2:
            print("Incorrectly Formatted Dataframe! Label column must have exactly 2 unique values.")
            return None
        
        '''Partition dataframe into two sets, one for each of the two unique values from the label column'''
        partition1 = df.loc[df[label_column] == label_values[0]]
        partition2 = df.loc[df[label_column] == label_values[1]]

        '''If no comparison columns specified, use all columns except the specified labed column'''
        if not comparison_columns:
            comparison_columns = list(df.columns)
            comparison_columns.remove(label_column)

        '''Determine the number of real valued columns on which we will do t-tests'''
        number_of_comparisons = len(comparison_columns)


        '''Store comparisons and p-values in two arrays'''
        comparisons = []
        pvals = []
        
        '''Loop through each comparison column, perform the t-test, and record the p-val'''
#         import pdb; pdb.set_trace()
        for column in comparison_columns:
            if len(partition1[column].dropna(axis=0)) <= mincount:
                continue
            elif len(partition2[column].dropna(axis=0)) <= mincount:
                continue
            else:
                stat, pval = scipy.stats.ttest_ind(partition1[column].dropna(axis=0), partition2[column].dropna(axis=0))
                comparisons.append(column)
                pvals.append(pval)
                
        '''Correct for multiple testing to determine if each comparison meets the new cutoff'''
        results = statsmodels.stats.multitest.multipletests(pvals=pvals, alpha=alpha, method=correction_method)
        reject = results[0]
        corrected_pval = results[1]

        '''Format results in a pandas dataframe'''
        results_df = pd.DataFrame(columns=['Comparison','P_Value'])

        '''If return all, add all comparisons and p-values to dataframe'''
        if return_all:
            if pval_return_corrected:
                for i in range(0,len(comparisons)):
                    results_df['Comparison'] = comparisons[i] ##fix
                    results_df['P_Value'] = corrected_pval[i]
                    results_df = results_df.append({'Comparison':comparisons[i],'P_Value':results[1][i]}, ignore_index=True)

            else:
                results_df['Comparison'] = comparisons
                results_df['P_Value'] = pvals

            '''Else only add significant comparisons'''
        else:
            for i in range(0, len(reject)):
                if reject[i]:
                    if pval_return_corrected:
                        results_df = results_df.append({'Comparison':comparisons[i],'P_Value':results[1][i]}, ignore_index=True)
                    else:
                        results_df = results_df.append({'Comparison':comparisons[i],'P_Value':pvals[i]}, ignore_index=True)


        '''Sort dataframe by ascending p-value'''
        results_df = results_df.sort_values(by='P_Value', ascending=True)
        results_df = results_df.reset_index(drop=True)

        '''If results df is not empty, return it, else return None'''
        if len(results_df) > 0:
            return results_df
        else:
            return None


    except:
        print("Incorrectly Formatted Dataframe!")
        return None


# Get phosphoproteomic data from paper

In [3]:
df = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable04.csv")
cols = [0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,27]
df = df.drop(df.columns[cols],axis=1)
df["phospho_site"] =  df["geneName"] + "_" + df["variableSites"]
df = df[pd.notnull(df['phospho_site'])]
df = df.drop(["variableSites","geneName"], axis = 1)
df1 = df.set_index("phospho_site")
df1.columns = df1.columns.str.replace(r'...TCGA', "", regex = True)
df1

,AO-A12D,C8-A131,AO-A12B,E2-A10A,BH-A18Q,C8-A130,C8-A138,E2-A154,BH-A0BZ,A8-A09I,...,AO-A12B,A2-A0SW,AO-A0JL,BH-A0BV,A2-A0YM,BH-A0C7,A2-A0SX,263d3f-I.CPTAC,blcdb9-I.CPTAC,c4155b-C.CPTAC
phospho_site,,,,,,,,,,,,,,,,,,,,,
AHNAK_S1007s,-0.919089,-2.565861,0.177221,0.203602,-0.086607,-2.700983,1.301494,2.952304,1.403537,0.161524,...,0.805357,4.389396,1.338505,3.450978,0.200324,1.728556,1.092898,-1.153228,0.234135,2.595100
AHNAK_S1010s,-1.271514,-1.726458,-0.320742,-0.523359,-0.315189,-2.208631,1.820337,3.049420,0.372156,0.039706,...,-0.124880,0.070029,0.495032,0.776968,1.017243,2.213385,0.444430,-0.331621,0.347756,2.527968
AHNAK_S1023s,NaN,NaN,NaN,NaN,NaN,NaN,-0.302638,3.157830,1.537787,0.238832,...,0.880172,0.812259,0.364182,1.626879,1.985442,-0.342381,1.767384,NaN,NaN,NaN
AHNAK_S1042s,-2.298519,-0.370305,-1.169382,0.330244,0.819633,-0.419166,1.570435,0.883498,-2.131714,-2.326377,...,-0.795385,0.797827,1.628386,1.382329,NaN,NaN,NaN,NaN,NaN,NaN
AHNAK_S1068s,-0.802335,-2.023904,-1.223738,-0.465794,-1.585542,-1.411369,1.394314,2.412517,1.060454,1.269601,...,-0.915370,2.065803,3.490470,2.797509,0.485597,2.001793,-0.352144,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GPR78_T210t,0.014945,2.483162,-0.324249,NaN,NaN,NaN,NaN,NaN,NaN,-3.572671,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SLC27A6_S217s T218t T224t,-0.698553,-1.693688,-0.175210,0.729908,0.613301,-0.054275,NaN,NaN,NaN,NaN,...,0.504689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C19ORF48_T47t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df2 = df1.T
df2.index.names = ['Sample ID']

df2.columns = df2.columns.str.replace(r' ', "_", regex = True)
df2

phospho_site,AHNAK_S1007s_,AHNAK_S1010s_,AHNAK_S1023s_,AHNAK_S1042s_,AHNAK_S1068s_,AHNAK_S106s_,AHNAK_S106s_S109s_,AHNAK_S1088s_,AHNAK_S110s_,AHNAK_S1123s_,...,BST1_S40s_,WNT9B_T234t_,C4ORF39_S92s_T107t_,AKD1_T1425t_,ZNF329_T359t_,GPR78_T210t_,SLC27A6_S217s_T218t_T224t_,C19ORF48_T47t_,PIGL_S250s_,RS1_Y165y_Y166y_T170t_
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-0.919089,-1.271514,NaN,-2.298519,-0.802335,-0.090999,-0.393695,0.378180,-0.374236,-1.777449,...,NaN,NaN,NaN,-7.085443,-3.870377,0.014945,-0.698553,NaN,NaN,NaN
C8-A131,-2.565861,-1.726458,NaN,-0.370305,-2.023904,0.637987,0.587573,0.693443,1.003493,-0.403074,...,NaN,NaN,NaN,1.676528,-3.110339,2.483162,-1.693688,NaN,NaN,NaN
AO-A12B,0.177221,-0.320742,NaN,-1.169382,-1.223738,0.059744,0.654143,0.028183,0.440229,1.103011,...,NaN,NaN,NaN,-0.213785,-2.580861,-0.324249,-0.175210,NaN,NaN,NaN
E2-A10A,0.203602,-0.523359,NaN,0.330244,-0.465794,0.540767,NaN,-4.867033,0.386164,0.754579,...,NaN,NaN,NaN,NaN,-0.449347,NaN,0.729908,NaN,NaN,NaN
BH-A18Q,-0.086607,-0.315189,NaN,0.819633,-1.585542,-0.909909,NaN,-1.963816,1.058329,2.100100,...,NaN,NaN,NaN,NaN,-2.060913,NaN,0.613301,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BH-A0C7,1.728556,2.213385,-0.342381,NaN,2.001793,1.182082,0.239081,NaN,0.925506,1.260388,...,NaN,-0.129123,NaN,NaN,-2.434978,NaN,NaN,NaN,NaN,NaN
A2-A0SX,1.092898,0.444430,1.767384,NaN,-0.352144,0.488462,1.204979,NaN,1.345080,0.780673,...,NaN,-2.493687,NaN,NaN,-2.523709,NaN,NaN,NaN,NaN,NaN
263d3f-I.CPTAC,-1.153228,-0.331621,NaN,NaN,NaN,NaN,NaN,-0.603446,0.103708,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Get mutation data

In [5]:
mutations = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable01.csv")
mutations = mutations.set_index("Sample ID")
mutations = mutations[["PIK3CA Mutation"]]
mutations = mutations.rename(columns = {"PIK3CA Mutation" : "PIK3CA_Mutation"})
mutations =  mutations.replace(np.nan, 'Wildtype', regex=True)
mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Missense_Mutation|Somatic|p.H1047R
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Missense_Mutation|Somatic|p.E545K
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [6]:
mutations.loc[mutations.PIK3CA_Mutation.str.contains('E542K')] = "Hotspot"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('E545K')] = "Hotspot"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('H1047R')] = "Hotspot"
mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Hotspot
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Hotspot
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [9]:
merged = pd.merge(df2, mutations, on = "Sample ID", how = "outer")
merged

,AHNAK_S1007s_,AHNAK_S1010s_,AHNAK_S1023s_,AHNAK_S1042s_,AHNAK_S1068s_,AHNAK_S106s_,AHNAK_S106s_S109s_,AHNAK_S1088s_,AHNAK_S110s_,AHNAK_S1123s_,...,WNT9B_T234t_,C4ORF39_S92s_T107t_,AKD1_T1425t_,ZNF329_T359t_,GPR78_T210t_,SLC27A6_S217s_T218t_T224t_,C19ORF48_T47t_,PIGL_S250s_,RS1_Y165y_Y166y_T170t_,PIK3CA_Mutation
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-0.919089,-1.271514,NaN,-2.298519,-0.802335,-0.090999,-0.393695,0.378180,-0.374236,-1.777449,...,NaN,NaN,-7.085443,-3.870377,0.014945,-0.698553,NaN,NaN,NaN,Wildtype
AO-A12D,0.029915,-1.190931,NaN,-2.049308,-1.141141,0.159369,-0.404252,0.631376,0.691124,-1.756544,...,-0.410227,2.292365,-5.464890,-1.487679,NaN,NaN,NaN,NaN,NaN,Wildtype
C8-A131,-2.565861,-1.726458,NaN,-0.370305,-2.023904,0.637987,0.587573,0.693443,1.003493,-0.403074,...,NaN,NaN,1.676528,-3.110339,2.483162,-1.693688,NaN,NaN,NaN,Missense_Mutation|Somatic|p.R108H
C8-A131,-1.847555,-1.732055,NaN,-0.080658,-2.201425,NaN,NaN,NaN,NaN,-1.407674,...,NaN,NaN,NaN,NaN,2.983764,NaN,NaN,NaN,NaN,Missense_Mutation|Somatic|p.R108H
AO-A12B,0.177221,-0.320742,NaN,-1.169382,-1.223738,0.059744,0.654143,0.028183,0.440229,1.103011,...,NaN,NaN,-0.213785,-2.580861,-0.324249,-0.175210,NaN,NaN,NaN,Wildtype
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BH-A0C7,1.728556,2.213385,-0.342381,NaN,2.001793,1.182082,0.239081,NaN,0.925506,1.260388,...,-0.129123,NaN,NaN,-2.434978,NaN,NaN,NaN,NaN,NaN,Wildtype
A2-A0SX,1.092898,0.444430,1.767384,NaN,-0.352144,0.488462,1.204979,NaN,1.345080,0.780673,...,-2.493687,NaN,NaN,-2.523709,NaN,NaN,NaN,NaN,NaN,Wildtype
263d3f-I.CPTAC,-1.153228,-0.331621,NaN,NaN,NaN,NaN,NaN,-0.603446,0.103708,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
wt_hotspot = merged.loc[(merged['PIK3CA_Mutation'] == "Wildtype") |
                (merged['PIK3CA_Mutation'] == "Hotspot")]

In [11]:
wt_hotspot = wt_hotspot.dropna(how = "all")
wt_hotspot

,AHNAK_S1007s_,AHNAK_S1010s_,AHNAK_S1023s_,AHNAK_S1042s_,AHNAK_S1068s_,AHNAK_S106s_,AHNAK_S106s_S109s_,AHNAK_S1088s_,AHNAK_S110s_,AHNAK_S1123s_,...,WNT9B_T234t_,C4ORF39_S92s_T107t_,AKD1_T1425t_,ZNF329_T359t_,GPR78_T210t_,SLC27A6_S217s_T218t_T224t_,C19ORF48_T47t_,PIGL_S250s_,RS1_Y165y_Y166y_T170t_,PIK3CA_Mutation
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-0.919089,-1.271514,NaN,-2.298519,-0.802335,-0.090999,-0.393695,0.378180,-0.374236,-1.777449,...,NaN,NaN,-7.085443,-3.870377,0.014945,-0.698553,NaN,NaN,NaN,Wildtype
AO-A12D,0.029915,-1.190931,NaN,-2.049308,-1.141141,0.159369,-0.404252,0.631376,0.691124,-1.756544,...,-0.410227,2.292365,-5.464890,-1.487679,NaN,NaN,NaN,NaN,NaN,Wildtype
AO-A12B,0.177221,-0.320742,NaN,-1.169382,-1.223738,0.059744,0.654143,0.028183,0.440229,1.103011,...,NaN,NaN,-0.213785,-2.580861,-0.324249,-0.175210,NaN,NaN,NaN,Wildtype
AO-A12B,0.805357,-0.124880,0.880172,-0.795385,-0.915370,0.455283,NaN,NaN,-0.582235,0.851940,...,NaN,NaN,NaN,-1.801849,NaN,0.504689,NaN,NaN,NaN,Wildtype
E2-A10A,0.203602,-0.523359,NaN,0.330244,-0.465794,0.540767,NaN,-4.867033,0.386164,0.754579,...,NaN,NaN,NaN,-0.449347,NaN,0.729908,NaN,NaN,NaN,Wildtype
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AO-A0JL,1.338505,0.495032,0.364182,1.628386,3.490470,NaN,NaN,NaN,0.078327,0.813096,...,NaN,0.857383,NaN,NaN,NaN,NaN,NaN,NaN,-1.528098,Wildtype
BH-A0BV,3.450978,0.776968,1.626879,1.382329,2.797509,NaN,NaN,NaN,0.289874,4.922284,...,NaN,1.678996,NaN,NaN,NaN,NaN,NaN,NaN,-0.247333,Hotspot
A2-A0YM,0.200324,1.017243,1.985442,NaN,0.485597,1.053262,2.380214,NaN,1.344298,1.975357,...,-1.012807,NaN,NaN,-1.485380,NaN,NaN,NaN,NaN,NaN,Wildtype


In [12]:


col_names = pd.Series(wt_hotspot.columns[:])
for dup in col_names[col_names.duplicated()].unique(): 
    col_names[col_names[col_names == dup].index.values.tolist()] = [dup + 'isoform_' + str(i) if i != 0 else dup for i in range(sum(col_names == dup))]
wt_hotspot.columns = col_names # rename the columns with the cols list


In [13]:
len(col_names)

54602

In [263]:
#wrap_ttest(wt_hotspot, "PIK3CA_Mutation", comparison_columns = cols , correction_method="fdr_bh", return_all=False, pval_return_corrected=False)

In [22]:
subset_test = wt_hotspot.iloc[:, list(range(10000)) + [-1]]
subset_test

,AHNAK_S1007s_,AHNAK_S1010s_,AHNAK_S1023s_,AHNAK_S1042s_,AHNAK_S1068s_,AHNAK_S106s_,AHNAK_S106s_S109s_,AHNAK_S1088s_,AHNAK_S110s_,AHNAK_S1123s_,...,C14ORF4_S659s_,C14ORF4_S662s_,C14ORF4_S69s_,C14ORF4_S705s_,C14ORF4_T231t_,C14ORF4_T286t_,C14ORF4_T610t_,C14ORF4_T610t_S622s_S639s_,C14ORF4_T610t_S639s_,PIK3CA_Mutation
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-0.919089,-1.271514,NaN,-2.298519,-0.802335,-0.090999,-0.393695,0.378180,-0.374236,-1.777449,...,-1.223948,NaN,-1.213137,2.190033,-2.443381,-0.521260,0.473313,NaN,0.497097,Wildtype
AO-A12D,0.029915,-1.190931,NaN,-2.049308,-1.141141,0.159369,-0.404252,0.631376,0.691124,-1.756544,...,-1.073427,NaN,-0.896175,-0.567563,0.529805,NaN,1.330425,NaN,NaN,Wildtype
AO-A12B,0.177221,-0.320742,NaN,-1.169382,-1.223738,0.059744,0.654143,0.028183,0.440229,1.103011,...,1.886776,NaN,-0.176964,0.377107,-0.115595,-0.876566,2.628460,NaN,2.300576,Wildtype
AO-A12B,0.805357,-0.124880,0.880172,-0.795385,-0.915370,0.455283,NaN,NaN,-0.582235,0.851940,...,1.454689,0.758775,1.110261,1.017096,NaN,NaN,2.926977,1.697482,2.677125,Wildtype
E2-A10A,0.203602,-0.523359,NaN,0.330244,-0.465794,0.540767,NaN,-4.867033,0.386164,0.754579,...,NaN,-5.848923,-3.524951,-0.082577,-0.952628,NaN,-1.077626,NaN,-0.582569,Wildtype
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AO-A0JL,1.338505,0.495032,0.364182,1.628386,3.490470,NaN,NaN,NaN,0.078327,0.813096,...,NaN,NaN,NaN,-2.826524,NaN,-3.794808,NaN,NaN,NaN,Wildtype
BH-A0BV,3.450978,0.776968,1.626879,1.382329,2.797509,NaN,NaN,NaN,0.289874,4.922284,...,NaN,NaN,NaN,-2.203730,NaN,-3.620915,NaN,NaN,NaN,Hotspot
A2-A0YM,0.200324,1.017243,1.985442,NaN,0.485597,1.053262,2.380214,NaN,1.344298,1.975357,...,NaN,NaN,NaN,0.533143,NaN,NaN,NaN,NaN,-3.241683,Wildtype


In [23]:
cols = list(subset_test.columns[:-1])
cols

['AHNAK_S1007s_',
 'AHNAK_S1010s_',
 'AHNAK_S1023s_',
 'AHNAK_S1042s_',
 'AHNAK_S1068s_',
 'AHNAK_S106s_',
 'AHNAK_S106s_S109s_',
 'AHNAK_S1088s_',
 'AHNAK_S110s_',
 'AHNAK_S1123s_',
 'AHNAK_S1135s_',
 'AHNAK_S1138s_',
 'AHNAK_S1158s_',
 'AHNAK_S115s_',
 'AHNAK_S1170s_',
 'AHNAK_S1196s_',
 'AHNAK_S1196s_isoform_1',
 'AHNAK_S1216s_',
 'AHNAK_S1263s_',
 'AHNAK_S1286s_',
 'AHNAK_S1298s_',
 'AHNAK_S1324s_',
 'AHNAK_S1344s_',
 'AHNAK_S135s_',
 'AHNAK_S135s_T146t_',
 'AHNAK_S1367s_',
 'AHNAK_S1391s_',
 'AHNAK_S1414s_',
 'AHNAK_M1441m_S1452s_',
 'AHNAK_S1452s_',
 'AHNAK_S1519s_',
 'AHNAK_S1542s_',
 'AHNAK_S1571s_M1572m_',
 'AHNAK_S1580s_',
 'AHNAK_S1600s_',
 'AHNAK_S1654s_',
 'AHNAK_S1674s_',
 'AHNAK_S1747s_',
 'AHNAK_S177s_',
 'AHNAK_S1782s_',
 'AHNAK_S1802s_',
 'AHNAK_S1876s_',
 'AHNAK_S1888s_',
 'AHNAK_S201s_',
 'AHNAK_S2080s_',
 'AHNAK_S2092s_',
 'AHNAK_S20s_',
 'AHNAK_S210s_',
 'AHNAK_S210s_S212s_S216s_',
 'AHNAK_S210s_S216s_',
 'AHNAK_S2138s_',
 'AHNAK_S2150s_',
 'AHNAK_S216s_',
 'AHNAK

In [25]:
u.wrap_ttest(subset_test, "PIK3CA_Mutation", comparison_columns = cols , correction_method="fdr_bh", return_all=True, pval_return_corrected=False)

,Comparison,P_Value
0,MYH14_S220s_,0.000205
1,NUMA1_S200s_,0.000252
2,MAPT_T111t_,0.000444
3,NCOR1_T1555t_,0.000675
4,SPTAN1_S1413s_,0.000920
...,...,...
6777,PARD3B_S1122s_,0.999392
6778,TENC1_S842s_S845s_,0.999601
6779,SYNPO2_S1181s_,0.999631
6780,KIAA1522_S386s_S389s_,0.999893


In [26]:
u.wrap_ttest(subset_test, "PIK3CA_Mutation", comparison_columns = cols , correction_method="fdr_bh", return_all=True, pval_return_corrected=True)

,Comparison,P_Value
0,NAV1_S954s_,0.808383
1,SYNPO_S804s_,0.808383
2,TRIM28_T541t_isoform_1,0.808383
3,TRIM28_T541t_,0.808383
4,SYNPO_S871s_,0.808383
...,...,...
6777,MDC1_S402s_T404t_,0.999926
6778,EIF5B_S186s_,0.999926
6779,AHNAK_S5310s_,0.999926
6780,NCL_S153s_M175m_,0.999958
